In [4]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import  sys
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis


In [5]:
!pip install pyLDAvis

Defaulting to user installation because normal site-packages is not writeable


In [6]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('C:/Users/moyka/OneDrive/Documents/GitHub/week0_starter_network_analysis')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [18]:
data_loader = SlackDataLoader(r"C:/Users/moyka/OneDrive/Documents/GitHub/anonymized/")

def get_channel_messages(channel):
    channel_messages = utils.get_messages_on_channel(f"C:/Users/moyka/OneDrive/Documents/GitHub//{channel}") 
    # Create an empty DataFrame
    df = pd.DataFrame(channel_messages)
    return df

def get_all_channels_message():
    dfs = []  # List to store individual DataFrames

    for channel in data_loader.channels:
        dfs.append(get_channel_messages(channel["name"]))

    # Concatenate all DataFrames into a single DataFrame
    result_df = pd.concat(dfs, ignore_index=True)

    return result_df

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/moyka/OneDrive/Documents/GitHub/anonymized/channels.json'

In [17]:
def preprocess_text(text):
    # Extract and remove URLs
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:
        text = text.replace(url, '')

    text = re.sub(r'<@.*?>', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Perform stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a string
    text = ' '.join(tokens)

    return text


In [15]:

def prepare_data(df):
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    sentence_list = [tweet for tweet in df['cleaned_text']]
    word_list = [sent.split() for sent in sentence_list]

    #Create dictionary which contains Id and word
    word_to_id = corpora.Dictionary(word_list) #generate unique tokens
    corpus = [word_to_id.doc2bow(tweet) for tweet in word_list]
    
    return df, word_list, word_to_id, corpus

In [14]:
def build_model(corpus, word_to_id):
    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                        id2word=word_to_id,
                                        num_topics=5,
                                        random_state=100,
                                        update_every=1,
                                        chunksize=100,
                                        passes=10,
                                        alpha='auto',
                                        per_word_topics=True)    
    return lda_model

In [11]:

def show_topics(lda_model):
    pprint(lda_model.show_topics(formatted=False))

In [10]:
def model_analysis(lda_model, corpus, word_list, word_to_id):
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))
    doc_lda = lda_model[corpus]


    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=word_to_id, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\n Lda model Coherence Score/Accuracy on Tweets: ', coherence_lda)

In [13]:
def get_top_topics(df):
    df, word_list, word_to_id, corpus = prepare_data(df)
    lda_model = build_model(corpus, word_to_id)

    # Show the top 10 topics
    show_topics(lda_model)
    

    # Visualize the top 10 topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, word_to_id)
    return LDAvis_prepared

### Top 10 topics of the different channels

In [8]:
df = get_channel_messages("all-week1")
get_top_topics(df)

NameError: name 'get_channel_messages' is not defined

In [39]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Eshetu\AppData\Roaming\nltk_data...


True

In [9]:
df = get_channel_messages("all-community-building")
get_top_topics(df)

NameError: name 'get_channel_messages' is not defined

### Top 10 topics of all channels

In [20]:
df = get_all_channels_message()
get_top_topics(df)

[(0,
  [('work', 0.07804269),
   ('instal', 0.039492182),
   ('get', 0.03234122),
   ('guy', 0.026092794),
   ('face', 0.02030699),
   ('return', 0.019842522),
   ('ok', 0.015252527),
   ('okay', 0.014948669),
   ('pip', 0.01389798),
   ('read', 0.013267612)]),
 (1,
  [('file', 0.0549693),
   ('line', 0.034151033),
   ('meet', 0.02210031),
   ('extract', 0.020621968),
   ('one', 0.020602355),
   ('good', 0.017431682),
   ('task', 0.017173653),
   ('imag', 0.015683454),
   ('logo', 0.01498308),
   ('need', 0.014000982)]),
 (2,
  [('channel', 0.18221219),
   ('join', 0.18198967),
   ('connect', 0.018376695),
   ('plea', 0.018241962),
   ('time', 0.01755661),
   ('start', 0.016417718),
   ('•', 0.014001578),
   ('today', 0.012244016),
   ('code', 0.012109588),
   ('link', 0.011688723)]),
 (3,
  [('thank', 0.041847672),
   ('think', 0.03968491),
   ('data', 0.03801513),
   ('sure', 0.021638291),
   ('creat', 0.019395646),
   ('right', 0.019231077),
   ('present', 0.01881592),
   ('dont', 0

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.371924 -0.020110       1        1  25.993380
4     -0.090644  0.267852       2        1  20.377566
1     -0.119150 -0.263536       3        1  19.558450
3     -0.070161  0.083032       4        1  18.391670
0     -0.091969 -0.067238       5        1  15.678934, topic_info=         Term         Freq        Total Category  logprob  loglift
0     channel  4785.000000  4785.000000  Default  30.0000  30.0000
1        join  4780.000000  4780.000000  Default  29.0000  29.0000
203      work  1237.000000  1237.000000  Default  28.0000  28.0000
24        use  1346.000000  1346.000000  Default  27.0000  27.0000
5632     file  1087.000000  1087.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
5079      fix    93.573706    94.353067   Topic5  -5.1316   1.8446
2138    frame   184.716212   186.261848   Topic5  -4.4515   1.8445
1393   recent    98.995630   100.062156   Topic5  -5.0753   1.8421
7526    conda   125.739371   131.182746   Topic5  -4.8361   1.8105
8245     sudo   123.883955   154.399777   Topic5  -4.8510   1.6327

[189 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
351       4  0.993177        abl
377       2  0.997747     access
2050      3  0.996749         ad
3254      4  0.991437    airflow
174       5  0.997243    alreadi
...     ...       ...        ...
429       4  0.994975       yeah
1131      2  0.994165  yesterday
810       5  0.995969       your
906       5  0.994548          ’
39        1  0.998721          •

[181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 4, 1])